In [ ]:
# Tejomaya_ETL_pipeline/
# │
# ├── agents/
# │   ├── searching_agent.py
# │   ├── parsing_agent.py
# │   ├── run_agents.py
# │
# ├── scheduler/
# │   └── scheduler.py
# │
# ├── api/
# │   └── main.py
# │
# ├── data/
# │   ├── output_excels/
# │   └── weekly_sebi_downloads.xlsx
# │
# ├── logs/
# │   ├── searching_agent.log
# │   ├── parsing_agent.log
# │   ├── scheduler.log
# │
# └── requirements.txt


# date - Monday to Sunday

In [27]:
import pandas as pd
from datetime import datetime, timedelta

excel_path = "Test_date.xlsx"
date_column = "Date"

today = datetime.today()

# Get THIS week's Monday (00:00)
this_monday = (today - timedelta(days=today.weekday())).replace(
    hour=0, minute=0, second=0, microsecond=0
)

weeks_back = 0   # <--- change this to check previous weeks

# Target week's Monday
target_monday = this_monday - timedelta(weeks=weeks_back)

# Target week's Sunday (23:59:59)
target_sunday = (target_monday + timedelta(days=6)).replace(
    hour=23, minute=59, second=59, microsecond=999999
)

# ❗ FIX: If this week → End at TODAY, not Sunday
if weeks_back == 0:
    target_sunday = today.replace(
        hour=23, minute=59, second=59, microsecond=999999
    )

print(f"Range for {weeks_back} week(s) back:", target_monday.date(), "to", target_sunday.date())

# Load data
df = pd.read_excel(excel_path)

# Filter
mask = (df[date_column] >= target_monday) & (df[date_column] <= target_sunday)
df_week = df[mask]

print(df_week)


Range for 0 week(s) back: 2025-11-17 to 2025-11-18
        Date  Yes/No
6 2025-11-17     NaN
7 2025-11-18     NaN


# scraping script

In [ ]:
# =========================================================================
# CRITICAL FIX FOR WINDOWS - MUST BE AT THE VERY TOP OF THE SCRIPT
# =========================================================================
import sys
import asyncio
import nest_asyncio
import platform
import os

if sys.platform.startswith("win"):
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    nest_asyncio.apply()
# =========================================================================

from urllib.parse import urljoin, urlparse
from crawl4ai import AsyncWebCrawler
from bs4 import BeautifulSoup
import aiohttp
import base64
from datetime import datetime
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ---------------- CONFIG -----------------
BASE_URL = "https://www.sebi.gov.in"
LISTING_URL = "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListing=yes&sid=6&ssid=23&smid=0"
CATEGORY = "SEBI"
SUBFOLDER = "Press Release"

if platform.system() == "Windows":
    BASE_PATH = r"C:\Users\Admin\Downloads\Tejomaya_pdfs\Akshayam Data"
else:
    BASE_PATH =   "/Users/admin/Downloads/Tejomaya_pdfs/Akshayam Data"

# ----------------------------------------

def ensure_year_month_structure(base_folder, category, subfolder, year, month_full):
    subfolder_path = os.path.join(base_folder, category, subfolder)
    year_path = os.path.join(subfolder_path, year)
    os.makedirs(year_path, exist_ok=True)
    month_path = os.path.join(year_path, month_full)
    os.makedirs(month_path, exist_ok=True)
    return month_path

def sanitize_filename(title):
    filename = re.sub(r'[\/\\\:\*\?"<>\|]', '_', title)
    filename = filename.replace(" ", "_")
    return filename + ".pdf"

async def download_pdf(session, pdf_url, save_path):
    filename = os.path.basename(urlparse(pdf_url).path)
    file_path = os.path.join(save_path, filename)
    if os.path.exists(file_path):
        print(f"⏭️ Skipping (already exists): {file_path}")
        return file_path
    
    try:
        async with session.get(pdf_url) as resp:
            if resp.status == 200:
                with open(file_path, "wb") as f:
                    f.write(await resp.read())
                print(f"📥 PDF downloaded: {file_path}")
                return file_path
            else:
                print(f"❌ Failed to download PDF: {pdf_url}, status: {resp.status}")
    except Exception as e:
        print(f"⚠️ Error downloading PDF: {e}")
    return None

def sync_fetch_links(page_number):
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)
    titles_and_urls = []
    try:
        driver.get(LISTING_URL)
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.points"))
        )

        if page_number > 1:
            page_index = page_number - 1
            driver.execute_script(f"searchFormNewsList('n', '{page_index}');")
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.points"))
            )

        soup = BeautifulSoup(driver.page_source, "html.parser")
        for link in soup.select("a.points[href]"):
            detail_url = link.get("href")
            title = link.get_text(strip=True)
            if detail_url:
                titles_and_urls.append({
                    "title": title,
                    "detail_url": urljoin(BASE_URL, detail_url)
                })
    finally:
        driver.quit()
    return titles_and_urls


async def scrape_sebi_page(page_number: int):
    downloaded_files = []
    print(f"\n--- Processing SEBI Page {page_number} ---")

    links_data = await asyncio.to_thread(sync_fetch_links, page_number)
    if not links_data:
        print(f"❌ No titles found on page {page_number}")
        return

    for link_data in links_data:
        title = link_data['title']
        detail_url = link_data['detail_url']
        print(f"\n🔗 Opening detail page: {detail_url}")

        async with AsyncWebCrawler() as crawler:
            detail_result = await crawler.arun(url=detail_url)
        soup_detail = BeautifulSoup(detail_result.html, "html.parser")

        # Extract date
        date_elem = soup_detail.select_one("h5")
        if date_elem:
            date_str = date_elem.get_text(strip=True)
            try:
                dt = datetime.strptime(date_str, "%b %d, %Y")
                year, month_full = str(dt.year), dt.strftime("%B")
            except ValueError:
                year, month_full = "UnknownYear", "UnknownMonth"
        else:
            date_str, year, month_full = "", "UnknownYear", "UnknownMonth"

        save_path = ensure_year_month_structure(BASE_PATH, CATEGORY, SUBFOLDER, year, month_full)

        # Try finding PDF (iframe or download button)
        pdf_url = None
        iframe = soup_detail.select_one("iframe")
        pdf_button = soup_detail.select_one("button#download")

        if iframe and "file=" in iframe.get("src", ""):
            pdf_url = iframe["src"].split("file=")[-1]
            if not pdf_url.startswith("http"):
                pdf_url = urljoin(BASE_URL, pdf_url)
            print("✅ PDF found in iframe:", pdf_url)
        elif pdf_button:
            pdf_url = detail_url.replace(".html", ".pdf")
            print("✅ PDF button found, trying direct PDF link:", pdf_url)
        else:
            print("⚠️ No direct PDF found, switching to print-to-PDF mode")

        # ---------------- Handle Download or Print ----------------
        file_path = None
        if pdf_url:
            async with aiohttp.ClientSession() as session:
                file_path = await download_pdf(session, pdf_url, save_path)

        if not file_path:  # fallback to print-to-PDF
            # Keep original page title or generate from URL for natural naming
            page_name = os.path.basename(urlparse(detail_url).path).replace(".html", ".pdf")
            file_path = os.path.join(save_path, page_name)

            options = webdriver.ChromeOptions()
            options.add_argument('--headless')
            driver = webdriver.Chrome(options=options)
            driver.get(detail_url)

            # Hide headers/sidebars and keep only main printable content
            driver.execute_script("""
                document.querySelectorAll('header, nav, aside, footer, .sidebar, .topnav').forEach(e => e.style.display='none');
                let main = document.querySelector('#printableArea');
                if(main) main.style.width = '100%';
            """)
            
            pdf = driver.execute_cdp_cmd("Page.printToPDF", {"printBackground": True})
            pdf_data = base64.b64decode(pdf['data'])
            with open(file_path, "wb") as f:
                f.write(pdf_data)
            driver.quit()
            print(f"🖨️ Printed to PDF (natural name): {file_path}")


        downloaded_files.append({
            "Title": title,
            "Date": date_str,
            "PDF_Path": os.path.abspath(file_path)
        })

    # Save Excel for the page
    if downloaded_files:
        excel_file = os.path.join(BASE_PATH, f"downloaded_pdfs_page{page_number}.xlsx")
        df = pd.DataFrame(downloaded_files)
        df.to_excel(excel_file, index=False)
        print(f"\n✅ Excel saved: {os.path.abspath(excel_file)}")
    else:
        print(f"No PDFs downloaded for page {page_number}, Excel not created.")


async def main():
    for page in range(1, 2):  # Change range as needed
        await scrape_sebi_page(page)

if __name__ == "__main__":
    asyncio.run(main())



# client summary

In [ ]:
import os
import re
import logging
from pathlib import Path
import pandas as pd
import nltk
from dotenv import load_dotenv
from unstructured.partition.pdf import partition_pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
from openpyxl import load_workbook, Workbook
import torch
import warnings
import time

# ---------------------- Setup ----------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)

warnings.filterwarnings("ignore", message="No languages specified, defaulting to English.")
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
load_dotenv()

# ---------------------- GPU Detection ----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["OLLAMA_USE_GPU"] = "1" if device.type == "cuda" else "0"
logging.info(f"💪 Using device: {device}")
logging.info("Ollama GPU status: " + ("Enabled" if os.environ.get("OLLAMA_USE_GPU") == "1" else "Disabled"))

# ---------------------- Initialize LLM ----------------------
llm = OllamaLLM(
    model="mistral:latest",
    device=device.type
)

chunk_prompt = PromptTemplate(
    template="""
    You are analyzing a regulatory or financial document.
    Summarize the following section in a **concise and factual** manner (2 sentences max).

    Focus only on:
    - The key regulatory or policy action (amendment, relaxation, proposal, or update)
    - Who or what it affects
    - Any critical dates or deadlines mentioned
    
    Avoid:
    - Legal citations, section numbers, or procedural details
    - Repetition of headers, page numbers, or contact information
    - Explanations, reasoning, or generic filler text

    Chunk Text:
    {text}

    Short factual summary:
    """,
    input_variables=["text"]
)



final_prompt = PromptTemplate(
    template="""
    You are a senior communications analyst summarizing official regulatory or legal documents
    for a client update. Create a concise, plain-language summary.

    Guidelines:
    - Limit strictly to 5–6 sentences.
    - Focus only on the core purpose, key changes, and who it applies to.
    - Avoid mentioning:
        • Emails, phone numbers, or submission instructions
        • Application processes, forms, or deadlines
        • Section or clause numbers unless critical for understanding
        • Legal or technical jargon (use simple terms instead)
    - Keep the tone professional and factual, not legalistic.
    - Do NOT repeat phrases or include disclaimers.

    Example Output:
    "SEBI has proposed amendments to the Listing Regulations to simplify compliance and ensure
    new securities issued through splits or schemes are in dematerialized form. The consultation
    seeks stakeholder feedback to improve market transparency and efficiency."

    Text:
    {text}

    Short Client Summary:
    """,
    input_variables=["text"]
)


# ---------------------- Folders ----------------------
OUTPUT_EXCEL_DIR = Path.cwd() / "output_excels"
OUTPUT_EXCEL_DIR.mkdir(exist_ok=True)
BASE_PDF_FOLDER_NAME = "All Data"

# ---------------------- PDF Cleaning ----------------------
def clean_pdf_text(text: str) -> str:
    """Remove headers, footers, page numbers, and repeated SEBI-like content."""
    # Split by page (form feed or manual detection)
    pages = re.split(r'\f+', text)
    cleaned_pages = []

    for page in pages:
        lines = page.splitlines()
        # Remove first 1–2 and last 1–2 lines (common headers/footers)
        if len(lines) > 4:
            lines = lines[2:-2]
        page_text = "\n".join(lines)

        # Remove page numbers like 'Page 3 of 10'
        page_text = re.sub(r'Page\s*\d+\s*(of\s*\d+)?', '', page_text, flags=re.IGNORECASE)

        # Remove repeated SEBI headers/footers
        page_text = re.sub(r'(Securities and Exchange Board of India|Consultation Paper|Master Circular)', '', page_text, flags=re.IGNORECASE)

        cleaned_pages.append(page_text.strip())

    # Merge pages
    cleaned_text = "\n".join(cleaned_pages)

    # Remove extra spaces/newlines
    cleaned_text = re.sub(r'\n{2,}', '\n', cleaned_text)
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text)

    return cleaned_text.strip()

# ---------------------- Language Detection ----------------------
def detect_language(text: str) -> str:
    devanagari_chars = re.findall(r'[\u0900-\u097F]', text)
    latin_chars = re.findall(r'[A-Za-z]', text)
    d_len, l_len = len(devanagari_chars), len(latin_chars)
    total = d_len + l_len
    if total == 0:
        return "unknown"
    hindi_ratio = d_len / total
    english_ratio = l_len / total
    if hindi_ratio > 0.3 and english_ratio > 0.3:
        return "mixed"
    elif hindi_ratio > 0.3:
        return "hindi"
    elif english_ratio > 0.3:
        return "english"
    else:
        return "unknown"

# ---------------------- English Filtering ----------------------
def filter_english_text(text: str) -> str:
    lines = text.split('\n')
    english_lines = [line for line in lines if re.search(r'[A-Za-z]', line) and not re.search(r'[\u0900-\u097F]', line)]
    return "\n".join(english_lines)

# ---------------------- Index Extraction ----------------------
def extract_indexing_from_first_page(pdf_path: str) -> str:
    try:
        first_page_data = partition_pdf(
            filename=str(pdf_path),
            strategy="fast",
            include_page_breaks=False,
            starting_page_number=1,
            max_pages=1
        )
        first_page_text = "\n".join(str(el) for el in first_page_data if el)
        #pattern = r'\b(?:REGD\.?\s*NO\.?\s*[A-Z.\s\-]*\d+(?:/\d+)*|IBBI/\d{4}-\d{2}/GN/REG\d{3})\b'
        pattern = r"""
        \b(
            REGD\.?\s*No\.?\s*[A-Z.\-\s]*\d+(?:/\d+)* |
            (?:CG|DL|MH|HR|UP|GJ|TN|RJ|KL|KA|WB|PB|CH|UK|AS|OR|BR|AP|TS|HP|GA|JK|NL|MN|TR|SK|AR)-[A-Z]{2}-E-\d{8}-\d+ |
            No\.?\s*[A-Z/.\-]*\d+(?:/\d+)* |
            SEBI/[A-Z]{2,}/\d{2}/\d{2} |
            S\.O\.\s*\d+\(E\) |
            IBBI/\d{4}-\d{2}/GN/REG\d+
        )\b
        """
        matches = re.findall(pattern, first_page_text, flags=re.VERBOSE)

        #matches = re.findall(pattern, first_page_text)
        return ", ".join(matches) if matches else "NA"
    except Exception as e:
        logging.warning(f"⚠️ Could not extract indexing info from {pdf_path}: {e}")
        return "NA"

# ---------------------- PDF Extraction ----------------------
def extract_pdf_text(pdf_path: str) -> (str, str, str):
    pdf_path = Path(pdf_path).resolve()
    indexing = extract_indexing_from_first_page(pdf_path)

    raw_data = partition_pdf(filename=str(pdf_path), strategy="fast", include_page_breaks=False)
    extracted_text = "\n".join(str(el) for el in raw_data if el).strip()

    if not extracted_text:
        logging.info(f"⚠️ No text found in fast extraction, using hi_res OCR for {pdf_path}")
        raw_data = partition_pdf(filename=str(pdf_path), strategy="hi_res", extract_images_in_pdf=True)
        extracted_text = "\n".join(str(el) for el in raw_data if el).strip()

    cleaned_text = clean_pdf_text(extracted_text)
    lang = detect_language(cleaned_text) if cleaned_text else "unknown"

    logging.info(f"🔤 Detected language: {lang}")
    logging.info(f"🆔 Indexing found: {indexing}")

    return cleaned_text, lang, indexing

# ---------------------- Summary Generation ----------------------
def generate_summary(extracted_text: str, max_tokens: int = 1000) -> dict:
    """Generate both embedding and client summaries."""
    try:
        splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=500)
        docs = splitter.create_documents([extracted_text])

        chunk_summaries = []
        for doc in docs:
            input_text = chunk_prompt.format(text=doc.page_content)
            out = llm.invoke(input_text).strip()
            if out:
                chunk_summaries.append(out)

        if not chunk_summaries:
            return {"embedding_text": "NA", "client_summary": "NA"}

        # Embedding text: dense factual content
        embedding_text = "\n".join(chunk_summaries)

        # Client summary: final high-level summary
        truncated_text = embedding_text[:max_tokens * 4]
        input_text_final = final_prompt.format(text=truncated_text)
        final_out = llm.invoke(input_text_final).strip()

        return {
            "embedding_text": embedding_text if embedding_text else "NA",
            "client_summary": final_out if final_out else "NA"
        }
    except Exception as e:
        logging.error(f"❌ LLM summarization failed: {e}")
        return {"embedding_text": "NA", "client_summary": "NA"}

# ---------------------- Excel Handling ----------------------
def update_excel(row: pd.Series):
    vertical = row["Verticals"]
    subcategory = row["SubCategory"]
    excel_path = OUTPUT_EXCEL_DIR / f"{vertical}.xlsx"

    if excel_path.exists():
        wb = load_workbook(excel_path)
    else:
        wb = Workbook()
        wb.remove(wb.active)

    if subcategory in wb.sheetnames:
        ws = wb[subcategory]
    else:
        ws = wb.create_sheet(title=subcategory)
        ws.append(list(row.index))

    ws.append([row[h] if row[h] != "" else "NA" for h in row.index])
    wb.save(excel_path)
    wb.close()
    logging.info(f"✅ Updated Excel: {excel_path}, Sheet: {subcategory}")

# ---------------------- Single PDF Processing ----------------------
def process_single_pdf(row: pd.Series):
    pdf_path = row["Path"]
    try:
        extracted_text, lang, indexing = extract_pdf_text(pdf_path)
        row["Indexing"] = indexing

        if extracted_text:
            if lang in ["english", "mixed"]:
                english_text = filter_english_text(extracted_text)
                summaries = generate_summary(english_text)
                row["Summary"] = summaries["client_summary"]
                row["EmbeddingText"] = summaries["embedding_text"]
            elif lang == "hindi":
                row["Summary"], row["EmbeddingText"] = "FULL_HINDI", "NA"
            else:
                row["Summary"], row["EmbeddingText"] = "NA", "NA"
        else:
            row["Summary"], row["EmbeddingText"] = "NA", "NA"
    except Exception as e:
        logging.error(f"❌ Error processing {pdf_path}: {e}")
        row["Summary"], row["EmbeddingText"], row["Indexing"] = "NA", "NA", "NA"
    return row

# ---------------------- Main Pipeline ----------------------
def main(excel_file: str):
    excel_file = Path(excel_file)
    if not excel_file.exists():
        raise FileNotFoundError(f"{excel_file} not found!")

    df = pd.read_excel(excel_file)
    required_cols = ["Verticals", "SubCategory", "Path"]
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Excel must have '{col}' column")

    if "Indexing" not in df.columns:
        df["Indexing"] = ""
    if "Summary" not in df.columns:
        df["Summary"] = ""
    if "EmbeddingText" not in df.columns:
        df["EmbeddingText"] = ""

    total = len(df)
    logging.info(f"🚀 Starting sequential PDF processing ({total} PDFs total)...")
    start_time = time.time()

    for idx, row in df.iterrows():
        pdf_start = time.time()
        logging.info(f"📘 Processing PDF {idx + 1}/{total}: {row['Path']}")
        processed_row = process_single_pdf(row)
        update_excel(processed_row)
        pdf_end = time.time()
        logging.info(f"⏱️ PDF {idx + 1} processed in {pdf_end - pdf_start:.2f} seconds")

    end_time = time.time()
    logging.info(f"🎉 All PDFs processed successfully in {end_time - start_time:.2f} seconds")

# ---------------------- Entry ----------------------
if __name__ == "__main__":
    import sys

    if len(sys.argv) < 2:
        print("❌ Usage: python client_summary.py <excel_file_path>")
        sys.exit(1)

    excel_file = sys.argv[1]

    print(f"\n🚀 Starting PDF summarization pipeline for: {excel_file}\n")
    logging.info(f"📂 Input Excel: {excel_file}")

    try:
        main(excel_file)
    except Exception as e:
        logging.error(f"❌ Fatal error in main(): {e}")


# Searching_agent.py

In [ ]:
# =========================================================================
# CRITICAL FIX FOR WINDOWS - MUST BE AT THE VERY TOP OF THE SCRIPT
# =========================================================================
import sys
import asyncio
import nest_asyncio
import platform
import os

if sys.platform.startswith("win"):
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    nest_asyncio.apply()
# =========================================================================

from urllib.parse import urljoin, urlparse
from crawl4ai import AsyncWebCrawler
from bs4 import BeautifulSoup
import aiohttp
import base64
from datetime import datetime, timedelta
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# -------- CONFIG --------
BASE_URL = "https://www.sebi.gov.in"
LISTING_URL = "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListing=yes&sid=6&ssid=23&smid=0"
CATEGORY = "SEBI"
SUBFOLDER = "Press Release"

if platform.system() == "Windows":
    BASE_PATH = r"C:\Users\Admin\Downloads\Tejomaya_pdfs\Akshayam Data"
else:
    BASE_PATH = "/Users/admin/Downloads/Tejomaya_pdfs/Akshayam Data"

# GLOBAL LIST FOR FINAL EXCEL
ALL_DOWNLOADED = []

# -------- WEEK RANGE LOGIC --------
def get_week_range(weeks_back=0):
    today = datetime.today()

    this_monday = (today - timedelta(days=today.weekday())).replace(
        hour=0, minute=0, second=0, microsecond=0
    )

    target_monday = this_monday - timedelta(weeks=weeks_back)

    target_sunday = (target_monday + timedelta(days=6)).replace(
        hour=23, minute=59, second=59, microsecond=999999
    )

    # Cap to today if checking THIS week
    if weeks_back == 0:
        target_sunday = today.replace(
            hour=23, minute=59, second=59, microsecond=999999
        )

    print(f"\n📅 Target range ({weeks_back} week(s) back): {target_monday.date()} → {target_sunday.date()}")
    return target_monday, target_sunday


# -------- HELPERS --------

def sanitize_filename(title):
    filename = re.sub(r'[\/\\\:\*\?"<>\|]', '_', title)
    filename = filename.replace(" ", "_")
    return filename + ".pdf"

def ensure_year_month_structure(base_folder, category, subfolder, year, month_full):
    subfolder_path = os.path.join(base_folder, category, subfolder)
    year_path = os.path.join(subfolder_path, year)
    os.makedirs(year_path, exist_ok=True)
    month_path = os.path.join(year_path, month_full)
    os.makedirs(month_path, exist_ok=True)
    return month_path

async def download_pdf(session, pdf_url, save_path):
    filename = os.path.basename(urlparse(pdf_url).path)
    file_path = os.path.join(save_path, filename)
    if os.path.exists(file_path):
        print(f"⏭️ Skipping (exists): {file_path}")
        return file_path
    
    try:
        async with session.get(pdf_url) as resp:
            if resp.status == 200:
                with open(file_path, "wb") as f:
                    f.write(await resp.read())
                print(f"📥 PDF downloaded: {file_path}")
                return file_path
            else:
                print(f"❌ Failed PDF download ({resp.status})")
    except Exception as e:
        print(f"⚠️ Error downloading PDF: {e}")
    return None

def sync_fetch_links(page_number):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)

    titles_and_urls = []
    try:
        driver.get(LISTING_URL)
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.points"))
        )

        if page_number > 1:
            page_index = page_number - 1
            driver.execute_script(f"searchFormNewsList('n', '{page_index}');")
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.points"))
            )

        soup = BeautifulSoup(driver.page_source, "html.parser")

        for link in soup.select("a.points[href]"):
            detail_url = link.get("href")
            title = link.get_text(strip=True)
            if detail_url:
                titles_and_urls.append({
                    "title": title,
                    "detail_url": urljoin(BASE_URL, detail_url)
                })
    finally:
        driver.quit()

    return titles_and_urls


# -------- MAIN SCRAPER --------

async def scrape_sebi_page(page_number: int, week_start, week_end):
    print(f"\n--- Processing SEBI Page {page_number} ---")

    links_data = await asyncio.to_thread(sync_fetch_links, page_number)
    if not links_data:
        print("❌ No titles found")
        return

    for link_data in links_data:
        title = link_data["title"]
        detail_url = link_data["detail_url"]

        print(f"\n🔗 Opening: {detail_url}")

        async with AsyncWebCrawler() as crawler:
            detail_result = await crawler.arun(url=detail_url)

        soup_detail = BeautifulSoup(detail_result.html, "html.parser")

        date_elem = soup_detail.select_one("h5")
        if not date_elem:
            continue

        date_str = date_elem.get_text(strip=True)
        try:
            dt = datetime.strptime(date_str, "%b %d, %Y")
        except:
            print("⚠️ Invalid date")
            continue

        # WEEK FILTER
        if not (week_start <= dt <= week_end):
            print(f"⏭️ Skipping (not in week): {dt.date()}")
            continue

        print(f"✅ MATCH (in week): {dt.date()}")

        year = str(dt.year)
        month_full = dt.strftime("%B")
        save_path = ensure_year_month_structure(BASE_PATH, CATEGORY, SUBFOLDER, year, month_full)

        # PDF extraction
        pdf_url = None
        iframe = soup_detail.select_one("iframe")
        pdf_button = soup_detail.select_one("button#download")

        if iframe and "file=" in iframe.get("src", ""):
            pdf_url = iframe["src"].split("file=")[-1]
            if not pdf_url.startswith("http"):
                pdf_url = urljoin(BASE_URL, pdf_url)

        elif pdf_button:
            pdf_url = detail_url.replace(".html", ".pdf")

        file_path = None

        if pdf_url:
            async with aiohttp.ClientSession() as session:
                file_path = await download_pdf(session, pdf_url, save_path)

        if not file_path:
            # Print-to-PDF fallback
            options = webdriver.ChromeOptions()
            options.add_argument("--headless")
            driver = webdriver.Chrome(options=options)
            driver.get(detail_url)

            pdf = driver.execute_cdp_cmd("Page.printToPDF", {"printBackground": True})
            driver.quit()

            pdf_data = base64.b64decode(pdf["data"])
            file_path = os.path.join(save_path, sanitize_filename(title))

            with open(file_path, "wb") as f:
                f.write(pdf_data)

            print(f"🖨️ Printed to PDF: {file_path}")

        # ADD TO GLOBAL LIST
        ALL_DOWNLOADED.append({
            "Verticals": CATEGORY,
            "SubCategory": SUBFOLDER,
            "Year": year,
            "Month": month_full,
            "File Name": os.path.basename(file_path),
            "Path": os.path.abspath(file_path)
        })


# -------- MAIN PROGRAM --------

async def main():
    weeks_back = 1   # 0=this week, 1=last week
    week_start, week_end = get_week_range(weeks_back)

    for page in range(1, 3):
        await scrape_sebi_page(page, week_start, week_end)

    # FINAL EXCEL
    if ALL_DOWNLOADED:
        df = pd.DataFrame(ALL_DOWNLOADED)
        script_dir = os.path.dirname(os.path.abspath(__file__))
        excel_output = os.path.join(script_dir, "weekly_sebi_downloads.xlsx")
        
        df.to_excel(excel_output, index=False)

        print("\n📄 FINAL EXCEL GENERATED:")
        print(excel_output)
    else:
        print("\n⚠️ No PDFs found for this week.")


if __name__ == "__main__":
    asyncio.run(main())


# parsing_agents.py

In [ ]:
import os
import re
import logging
from pathlib import Path
import pandas as pd
import nltk
from dotenv import load_dotenv
from unstructured.partition.pdf import partition_pdf
from langchain.text_splitter import RecursiveCharacterTextSplitter

# from langchain_ollama import OllamaLLM
from langchain.llms import Ollama

from langchain.prompts import PromptTemplate
from openpyxl import load_workbook, Workbook
import torch
import warnings
import time

# ---------------------- Setup ----------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.StreamHandler()]
)

warnings.filterwarnings("ignore", message="No languages specified, defaulting to English.")
nltk.download('punkt', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)
load_dotenv()

# ---------------------- GPU Detection ----------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["OLLAMA_USE_GPU"] = "1" if device.type == "cuda" else "0"
logging.info(f"💪 Using device: {device}")
logging.info("Ollama GPU status: " + ("Enabled" if os.environ.get("OLLAMA_USE_GPU") == "1" else "Disabled"))

# ---------------------- Initialize LLM ----------------------
# llm = OllamaLLM(
#     model="mistral:latest",
#     device=device.type
# )
from langchain.llms import Ollama

llm = Ollama(model="mistral:latest")


chunk_prompt = PromptTemplate(
    template="""
    You are analyzing a regulatory or financial document.
    Summarize the following section in a **concise and factual** manner (2 sentences max).

    Focus only on:
    - The key regulatory or policy action (amendment, relaxation, proposal, or update)
    - Who or what it affects
    - Any critical dates or deadlines mentioned
    
    Avoid:
    - Legal citations, section numbers, or procedural details
    - Repetition of headers, page numbers, or contact information
    - Explanations, reasoning, or generic filler text

    Chunk Text:
    {text}

    Short factual summary:
    """,
    input_variables=["text"]
)



final_prompt = PromptTemplate(
    template="""
    You are a senior communications analyst summarizing official regulatory or legal documents
    for a client update. Create a concise, plain-language summary.

    Guidelines:
    - Limit strictly to 5–6 sentences.
    - Focus only on the core purpose, key changes, and who it applies to.
    - Avoid mentioning:
        • Emails, phone numbers, or submission instructions
        • Application processes, forms, or deadlines
        • Section or clause numbers unless critical for understanding
        • Legal or technical jargon (use simple terms instead)
    - Keep the tone professional and factual, not legalistic.
    - Do NOT repeat phrases or include disclaimers.

    Example Output:
    "SEBI has proposed amendments to the Listing Regulations to simplify compliance and ensure
    new securities issued through splits or schemes are in dematerialized form. The consultation
    seeks stakeholder feedback to improve market transparency and efficiency."

    Text:
    {text}

    Short Client Summary (entire answer MUST be inside double quotes):
    "{{ summary }}"

    """,
    input_variables=["text"]
)


# ---------------------- Folders ----------------------
OUTPUT_EXCEL_DIR = Path.cwd() / "output_excels"
OUTPUT_EXCEL_DIR.mkdir(exist_ok=True)
BASE_PDF_FOLDER_NAME = "All Data"

# ---------------------- PDF Cleaning ----------------------
def clean_pdf_text(text: str) -> str:
    """Remove headers, footers, page numbers, and repeated SEBI-like content."""
    # Split by page (form feed or manual detection)
    pages = re.split(r'\f+', text)
    cleaned_pages = []

    for page in pages:
        lines = page.splitlines()
        # Remove first 1–2 and last 1–2 lines (common headers/footers)
        if len(lines) > 4:
            lines = lines[2:-2]
        page_text = "\n".join(lines)

        # Remove page numbers like 'Page 3 of 10'
        page_text = re.sub(r'Page\s*\d+\s*(of\s*\d+)?', '', page_text, flags=re.IGNORECASE)

        # Remove repeated SEBI headers/footers
        page_text = re.sub(r'(Securities and Exchange Board of India|Consultation Paper|Master Circular)', '', page_text, flags=re.IGNORECASE)

        cleaned_pages.append(page_text.strip())

    # Merge pages
    cleaned_text = "\n".join(cleaned_pages)

    # Remove extra spaces/newlines
    cleaned_text = re.sub(r'\n{2,}', '\n', cleaned_text)
    cleaned_text = re.sub(r'\s{2,}', ' ', cleaned_text)

    return cleaned_text.strip()

# ---------------------- Language Detection ----------------------
def detect_language(text: str) -> str:
    devanagari_chars = re.findall(r'[\u0900-\u097F]', text)
    latin_chars = re.findall(r'[A-Za-z]', text)
    d_len, l_len = len(devanagari_chars), len(latin_chars)
    total = d_len + l_len
    if total == 0:
        return "unknown"
    hindi_ratio = d_len / total
    english_ratio = l_len / total
    if hindi_ratio > 0.3 and english_ratio > 0.3:
        return "mixed"
    elif hindi_ratio > 0.3:
        return "hindi"
    elif english_ratio > 0.3:
        return "english"
    else:
        return "unknown"

# ---------------------- English Filtering ----------------------
def filter_english_text(text: str) -> str:
    lines = text.split('\n')
    english_lines = [line for line in lines if re.search(r'[A-Za-z]', line) and not re.search(r'[\u0900-\u097F]', line)]
    return "\n".join(english_lines)

# ---------------------- Index Extraction ----------------------
def extract_indexing_from_first_page(pdf_path: str) -> str:
    try:
        first_page_data = partition_pdf(
            filename=str(pdf_path),
            strategy="fast",
            include_page_breaks=False,
            starting_page_number=1,
            max_pages=1
        )
        first_page_text = "\n".join(str(el) for el in first_page_data if el)
        #pattern = r'\b(?:REGD\.?\s*NO\.?\s*[A-Z.\s\-]*\d+(?:/\d+)*|IBBI/\d{4}-\d{2}/GN/REG\d{3})\b'
        pattern = r"""
        \b(
            REGD\.?\s*No\.?\s*[A-Z.\-\s]*\d+(?:/\d+)* |
            (?:CG|DL|MH|HR|UP|GJ|TN|RJ|KL|KA|WB|PB|CH|UK|AS|OR|BR|AP|TS|HP|GA|JK|NL|MN|TR|SK|AR)-[A-Z]{2}-E-\d{8}-\d+ |
            No\.?\s*[A-Z/.\-]*\d+(?:/\d+)* |
            SEBI/[A-Z]{2,}/\d{2}/\d{2} |
            S\.O\.\s*\d+\(E\) |
            IBBI/\d{4}-\d{2}/GN/REG\d+
        )\b
        """
        matches = re.findall(pattern, first_page_text, flags=re.VERBOSE)

        #matches = re.findall(pattern, first_page_text)
        return ", ".join(matches) if matches else "NA"
    except Exception as e:
        logging.warning(f"⚠️ Could not extract indexing info from {pdf_path}: {e}")
        return "NA"

# ---------------------- PDF Extraction ----------------------
def extract_pdf_text(pdf_path: str) -> (str, str, str):
    pdf_path = Path(pdf_path).resolve()
    indexing = extract_indexing_from_first_page(pdf_path)

    raw_data = partition_pdf(filename=str(pdf_path), strategy="fast", include_page_breaks=False)
    extracted_text = "\n".join(str(el) for el in raw_data if el).strip()

    if not extracted_text:
        logging.info(f"⚠️ No text found in fast extraction, using hi_res OCR for {pdf_path}")
        raw_data = partition_pdf(filename=str(pdf_path), strategy="hi_res", extract_images_in_pdf=True)
        extracted_text = "\n".join(str(el) for el in raw_data if el).strip()

    cleaned_text = clean_pdf_text(extracted_text)
    lang = detect_language(cleaned_text) if cleaned_text else "unknown"

    logging.info(f"🔤 Detected language: {lang}")
    logging.info(f"🆔 Indexing found: {indexing}")

    return cleaned_text, lang, indexing

# ---------------------- Summary Generation ----------------------
def generate_summary(extracted_text: str, max_tokens: int = 1000) -> dict:
    """Generate both embedding and client summaries."""
    try:
        splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=500)
        docs = splitter.create_documents([extracted_text])

        chunk_summaries = []
        for doc in docs:
            input_text = chunk_prompt.format(text=doc.page_content)
            out = llm.invoke(input_text).strip()
            if out:
                chunk_summaries.append(out)

        if not chunk_summaries:
            return {"embedding_text": "NA", "client_summary": "NA"}

        # Embedding text: dense factual content
        embedding_text = "\n".join(chunk_summaries)

        # Client summary: final high-level summary
        truncated_text = embedding_text[:max_tokens * 4]
        input_text_final = final_prompt.format(text=truncated_text)
        final_out = llm.invoke(input_text_final).strip()

        return {
            "embedding_text": embedding_text if embedding_text else "NA",
            "client_summary": final_out if final_out else "NA"
        }
    except Exception as e:
        logging.error(f"❌ LLM summarization failed: {e}")
        return {"embedding_text": "NA", "client_summary": "NA"}

# ---------------------- Excel Handling ----------------------
def update_excel(row: pd.Series):
    vertical = row["Verticals"]
    subcategory = row["SubCategory"]
    excel_path = OUTPUT_EXCEL_DIR / f"{vertical}.xlsx"

    if excel_path.exists():
        wb = load_workbook(excel_path)
    else:
        wb = Workbook()
        wb.remove(wb.active)

    if subcategory in wb.sheetnames:
        ws = wb[subcategory]
    else:
        ws = wb.create_sheet(title=subcategory)
        ws.append(list(row.index))

    ws.append([row[h] if row[h] != "" else "NA" for h in row.index])
    wb.save(excel_path)
    wb.close()
    logging.info(f"✅ Updated Excel: {excel_path}, Sheet: {subcategory}")

# ---------------------- Single PDF Processing ----------------------
def process_single_pdf(row: pd.Series):
    pdf_path = row["Path"]
    try:
        extracted_text, lang, indexing = extract_pdf_text(pdf_path)
        row["Indexing"] = indexing

        if extracted_text:
            if lang in ["english", "mixed"]:
                english_text = filter_english_text(extracted_text)
                summaries = generate_summary(english_text)
                row["Summary"] = summaries["client_summary"]
                row["EmbeddingText"] = summaries["embedding_text"]
            elif lang == "hindi":
                row["Summary"], row["EmbeddingText"] = "FULL_HINDI", "NA"
            else:
                row["Summary"], row["EmbeddingText"] = "NA", "NA"
        else:
            row["Summary"], row["EmbeddingText"] = "NA", "NA"
    except Exception as e:
        logging.error(f"❌ Error processing {pdf_path}: {e}")
        row["Summary"], row["EmbeddingText"], row["Indexing"] = "NA", "NA", "NA"
    return row

# ---------------------- Main Pipeline ----------------------
def main(excel_file: str):
    excel_file = Path(excel_file)
    if not excel_file.exists():
        raise FileNotFoundError(f"{excel_file} not found!")

    df = pd.read_excel(excel_file)
    required_cols = ["Verticals", "SubCategory", "Path"]
    for col in required_cols:
        if col not in df.columns:
            raise ValueError(f"Excel must have '{col}' column")

    if "Indexing" not in df.columns:
        df["Indexing"] = ""
    if "Summary" not in df.columns:
        df["Summary"] = ""
    if "EmbeddingText" not in df.columns:
        df["EmbeddingText"] = ""

    total = len(df)
    logging.info(f"🚀 Starting sequential PDF processing ({total} PDFs total)...")
    start_time = time.time()

    for idx, row in df.iterrows():
        pdf_start = time.time()
        logging.info(f"📘 Processing PDF {idx + 1}/{total}: {row['Path']}")
        processed_row = process_single_pdf(row)
        update_excel(processed_row)
        pdf_end = time.time()
        logging.info(f"⏱️ PDF {idx + 1} processed in {pdf_end - pdf_start:.2f} seconds")

    end_time = time.time()
    logging.info(f"🎉 All PDFs processed successfully in {end_time - start_time:.2f} seconds")

# ---------------------- Entry ----------------------
if __name__ == "__main__":
    import sys

    # Auto-detect the excel from scraper output
    default_excel = Path.cwd() / "weekly_sebi_downloads.xlsx"

    if len(sys.argv) >= 2:
        excel_file = Path(sys.argv[1])
        print(f"📂 Using Excel from CLI: {excel_file}")
    elif default_excel.exists():
        excel_file = default_excel
        print(f"📂 Auto-detected Excel: {excel_file}")
    else:
        print("❌ No Excel file provided and weekly_sebi_downloads.xlsx not found!")
        print("Usage: python process_pdfs.py <excel_file_path>")
        sys.exit(1)


    print(f"\n🚀 Starting PDF summarization pipeline for: {excel_file}\n")
    logging.info(f"📂 Input Excel: {excel_file}")

    try:
        main(excel_file)
    except Exception as e:
        logging.error(f"❌ Fatal error in main(): {e}")


# run_agents.py

In [ ]:
#!/usr/bin/env python
import subprocess
import time
import logging
from pathlib import Path
import sys

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s %(message)s"
)

PYTHON = sys.executable
SEARCH_AGENT = str(Path("Searching_agent.py").resolve())
PARSE_AGENT = str(Path("Parsing_agent.py").resolve())

SEARCH_LOG = str(Path("searching_agent.log").resolve())
PARSE_LOG = str(Path("parsing_agent.log").resolve())

EXCEL_OUTPUT = Path("weekly_sebi_downloads.xlsx")


def run_process(script_path, log_path):
    """Runs a script and logs output to file."""
    logging.info(f"🚀 Starting script: {script_path}")
    with open(log_path, "a") as f:
        f.write(f"\n\n=== RUN AT {time.strftime('%Y-%m-%d %H:%M:%S')} ===\n")
        result = subprocess.run([PYTHON, script_path], stdout=f, stderr=f)
    logging.info(f"✅ Finished {script_path} with returncode={result.returncode}")
    return result.returncode


def main():
    logging.info("=============== STARTING AGENT CHAIN ===============")

    # 1️⃣ Run Searching Agent
    rc1 = run_process(SEARCH_AGENT, SEARCH_LOG)
    if rc1 != 0:
        logging.error("❌ Searching Agent failed. Stopping chain.")
        return

    # 2️⃣ Check if Excel exists
    if not EXCEL_OUTPUT.exists():
        logging.error(f"❌ Excel not found: {EXCEL_OUTPUT}. Cannot run Parsing Agent.")
        return

    logging.info(f"📄 Excel found --> {EXCEL_OUTPUT}. Proceeding to Parsing Agent.")

    # 3️⃣ Run Parsing Agent
    rc2 = run_process(PARSE_AGENT, PARSE_LOG)
    if rc2 != 0:
        logging.error("❌ Parsing Agent failed!")
        return

    logging.info("🎉 BOTH AGENTS COMPLETED SUCCESSFULLY!")


if __name__ == "__main__":
    main()


# scheduler.py

In [ ]:
#!/usr/bin/env python
import schedule
import time
import subprocess
from pathlib import Path
import sys
import logging

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

# ------------- CONFIG -------------
# If you want the scheduler to use the same Python interpreter you launched it with,
# leave PYTHON_PATH = sys.executable. Otherwise set explicit interpreter path.
PYTHON_PATH = sys.executable  # uses the current Python (recommended)
SCRIPT_NAME = "Searching_agent.py"  # <-- change to "Parsing_agent.py" if needed

SCRIPT_PATH = str(Path(__file__).parent.joinpath(SCRIPT_NAME).resolve())
# LOG file for agent stdout/stderr
LOG_PATH = str(Path(__file__).parent.joinpath("agent_run.log").resolve())
# ----------------------------------

def run_agent():
    logging.info("Starting agent: %s", SCRIPT_PATH)
    # Use subprocess.run to wait and capture return code; redirect stdout/stderr to log file
    with open(LOG_PATH, "a") as f:
        f.write("\n\n=== Run at: {} ===\n".format(time.strftime("%Y-%m-%d %H:%M:%S")))
        try:
            # On Windows, shell=False; provide full interpreter + script
            result = subprocess.run([PYTHON_PATH, SCRIPT_PATH], stdout=f, stderr=f, check=False)
            logging.info("Agent finished with returncode=%s", result.returncode)
        except Exception as e:
            logging.exception("Failed to run agent: %s", e)

# Examples: schedule as needed
# schedule.every().day.at("09:00").do(run_agent)           # daily 09:00

# schedule.every().day.at("12:59").do(run_agent)

def run_agents():
    subprocess.Popen([sys.executable, "run_agents.py"])

schedule.every().day.at("13:12").do(run_agents)

# schedule.every().monday.at("07:00").do(run_agent)     # weekly Monday 07:00
# schedule.every().hour.do(run_agent)                   # every hour

logging.info("Scheduler started. Agent script: %s", SCRIPT_PATH)

try:
    while True:
        schedule.run_pending()
        time.sleep(1)
except KeyboardInterrupt:
    logging.info("Scheduler stopped by user.")
